**Modulo 3: TensorFlow y Keras para Neural Networks**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Parte 3.3: Guardar y cargar una red neuronal de Keras

Las redes neuronales complejas tardarán mucho tiempo en adaptarse/entrenarse. Es útil poder guardar estas redes neuronales para que puedan recargarse más tarde. Una red neuronal recargada no requerirá un nuevo entrenamiento. Keras proporciona tres formatos para guardar redes neuronales.

* **YAML** - Almacena la estructura de la red neuronal (sin pesos) en el [YAML file format](https://en.wikipedia.org/wiki/YAML).
* **JSON** - Almacena la estructura de la red neuronal (sin pesos) en el [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Almacena la red neuronal completa (con pesos) en el [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). No confunda HDF5 con [HDFS](https://en.wikipedia.org/wiki/Apache_Hadoop).

Por lo general, querrá guardar en HDF5.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

save_path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Tratado de missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas a Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regresión

# construcción de la neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Oculta 1
model.add(Dense(10, activation='relu')) # Oculta 2
model.add(Dense(1)) # Salida
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

# Predict
pred = model.predict(x)

# RMSE error.  
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Antes de Guardar el (RMSE): {score}")

# guardar la estructura de la red neuronal en JSON (sin weights)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# guardar toda la red en HDF5 (guardar todo es lo recomendado)
model.save(os.path.join(save_path,"network.h5"))

Epoch 1/100
13/13 - 0s - loss: 204515.2344 - 245ms/epoch - 19ms/step
Epoch 2/100
13/13 - 0s - loss: 7687.1880 - 37ms/epoch - 3ms/step
Epoch 3/100
13/13 - 0s - loss: 11632.9727 - 34ms/epoch - 3ms/step
Epoch 4/100
13/13 - 0s - loss: 1722.8514 - 33ms/epoch - 3ms/step
Epoch 5/100
13/13 - 0s - loss: 1582.1915 - 34ms/epoch - 3ms/step
Epoch 6/100
13/13 - 0s - loss: 910.1257 - 33ms/epoch - 3ms/step
Epoch 7/100
13/13 - 0s - loss: 852.8941 - 32ms/epoch - 2ms/step
Epoch 8/100
13/13 - 0s - loss: 787.2728 - 36ms/epoch - 3ms/step
Epoch 9/100
13/13 - 0s - loss: 749.2866 - 33ms/epoch - 3ms/step
Epoch 10/100
13/13 - 0s - loss: 687.3815 - 31ms/epoch - 2ms/step
Epoch 11/100
13/13 - 0s - loss: 641.3677 - 32ms/epoch - 2ms/step
Epoch 12/100
13/13 - 0s - loss: 596.4457 - 31ms/epoch - 2ms/step
Epoch 13/100
13/13 - 0s - loss: 553.3340 - 31ms/epoch - 2ms/step
Epoch 14/100
13/13 - 0s - loss: 500.1074 - 32ms/epoch - 2ms/step
Epoch 15/100
13/13 - 0s - loss: 460.6832 - 30ms/epoch - 2ms/step
Epoch 16/100
13/13 - 0s 

El siguiente código configura una red neuronal y lee los datos (para predicciones), pero no borra el directorio del modelo ni se ajusta a la red neuronal. Se utilizan los pesos del ajuste anterior.

Ahora recargamos la red y realizamos otra predicción. El RMSE debería coincidir exactamente con el anterior si la red neuronal realmente se guardó y recargó.

In [3]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
# RMSE error.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Despues de cargar el modelo (RMSE): {score}")

Despues de cargar el modelo (RMSE): 4.096458657364227
